In [31]:
import pandas as pd

###승희한테 코드받아서
감사의견 라벨링
자본잠식 라벨링
이자보상배율 라벨링

라벨링하고 x,y,z체크

부실 이후 데이터 삭제



In [32]:
df = pd.read_csv('../dataset2/data2A_첫행날리기전.csv',index_col=0)

In [33]:
df

,회사명,거래소코드,회계년도,감사의견코드,산업코드,산업명,설립일,기업규모코드,기업규모명,통계청 한국표준산업분류 코드 10차(대분류),...,이자보상배율,new_총자본증가율,new_총부채증가율,new_순이익증가율,new_유형자산증가율,new_유동자산증가율,new_영업이익증가율,new_재고자산증가율,new_매출액증가율,new_비유동자산증가율
70560,(주)포스코건설,10000,2012-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,1479.098523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70561,(주)포스코건설,10000,2013-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,1384.016443,2.842294,-22.408393,-71.478710,40.756393,-18.258059,13.631720,-16.164419,14.016854,4.081387
70562,(주)포스코건설,10000,2014-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,633.408037,0.143478,-7.484781,-86.472926,5.344684,-12.449193,-31.415835,-8.092776,-6.396890,17.867219
70563,(주)포스코건설,10000,2015-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,733.133307,14.401493,-4.837130,127.038293,-4.509898,6.351912,-11.590310,-34.770304,-13.012219,-1.042386
70564,(주)포스코건설,10000,2016-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-808.461957,-23.096221,10.876742,-2616.490936,-10.676382,-5.085581,-173.802637,54.677241,-15.921085,-8.134513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221455,주식회사두루안,101635,2021-12-01,UQ,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,215.801478,26.187846,-3.359280,317.304431,-1.374490,58.085287,-7.162331,7.322329,1.872254,-1.302387
221456,주식회사두루안,101635,2022-12-01,UQ,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,-421.159620,10.028442,36.617113,-51.677152,54.399781,10.261314,-403.105893,63.082647,-12.227925,27.676594
16765,넬코리아주식회사,101667,2020-12-01,NaN,64102,토목 건설업,2017/08/08,20.0,중소기업,41,...,-1358.509774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16766,넬코리아주식회사,101667,2021-12-01,QS,64102,토목 건설업,2017/08/08,20.0,중소기업,41,...,-1111.179691,408.324797,20.291910,190.228461,13.969112,-23.104733,173.681224,-28.716629,-14.067465,64.013882


### 감사의견코드 라벨링
부도 : A
정상 : a

In [34]:
# Step 2 : 회계년도를 datetime
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y-%m-%d')
df_sorted = df.sort_values(by=['거래소코드', '회계년도'])

# Step 3 : 감사의견코드 중 DI, DS DU, AG, QS가 존재하면 A 아니면 a
df_sorted['감사의견_y'] = df_sorted['감사의견코드'].apply(lambda x: "A" if x in ['DI', 'DS', 'DU', 'AG', 'QS'] else "a")


### 자본잠식률 100% 라벨링
부도 : B
정상 : b

In [35]:
# Step 3 : 자본잠식률 100이면 B 아니면 b
df_sorted['자본잠식률_100'] = df_sorted['자본잠식률'].apply(lambda x: "B" if x >= 100 else "b")

df_sorted['자본잠식률_100']

70560     b
70561     b
70562     b
70563     b
70564     b
         ..
221455    b
221456    b
16765     B
16766     B
16767     B
Name: 자본잠식률_100, Length: 235434, dtype: object

### 자본잠식률 50% 2년 연속
부도 : C
정상 : c

In [36]:
# 데이터셋을 거래소코드와 회계년도 기준으로 정렬
df_sorted.sort_values(by=['거래소코드', '회계년도'], inplace=True)

# 새로운 'Label' 컬럼을 기본값 'c'로 추가
df_sorted['자본잠식률_50'] = 'c'

# 이전 행의 값 초기화
prev_code = None
prev_cap_erode_rate = None

# 각 행을 순회하며 조건에 따라 'Label' 컬럼 설정
for index, row in df_sorted.iterrows():
    current_code = row['거래소코드']
    current_cap_erode_rate = row['자본잠식률']

    # 현재 행과 이전 행이 같은 회사이고, 두 해 모두 자본잠식률이 50 이상인 경우 'C'로 표시
    if current_code == prev_code and current_cap_erode_rate >= 50 and prev_cap_erode_rate >= 50:
        df_sorted.at[index, '자본잠식률_50'] = 'C'

    # 이전 행 값 업데이트
    prev_code = current_code
    prev_cap_erode_rate = current_cap_erode_rate

# 최종 데이터셋 미리보기
df_sorted.head()


,회사명,거래소코드,회계년도,감사의견코드,산업코드,산업명,설립일,기업규모코드,기업규모명,통계청 한국표준산업분류 코드 10차(대분류),...,new_순이익증가율,new_유형자산증가율,new_유동자산증가율,new_영업이익증가율,new_재고자산증가율,new_매출액증가율,new_비유동자산증가율,감사의견_y,자본잠식률_100,자본잠식률_50
70560,(주)포스코건설,10000,2012-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c
70561,(주)포스코건설,10000,2013-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-71.478710,40.756393,-18.258059,13.631720,-16.164419,14.016854,4.081387,a,b,c
70562,(주)포스코건설,10000,2014-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-86.472926,5.344684,-12.449193,-31.415835,-8.092776,-6.396890,17.867219,a,b,c
70563,(주)포스코건설,10000,2015-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,127.038293,-4.509898,6.351912,-11.590310,-34.770304,-13.012219,-1.042386,a,b,c
70564,(주)포스코건설,10000,2016-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-2616.490936,-10.676382,-5.085581,-173.802637,54.677241,-15.921085,-8.134513,a,b,c


In [37]:
df_sorted

,회사명,거래소코드,회계년도,감사의견코드,산업코드,산업명,설립일,기업규모코드,기업규모명,통계청 한국표준산업분류 코드 10차(대분류),...,new_순이익증가율,new_유형자산증가율,new_유동자산증가율,new_영업이익증가율,new_재고자산증가율,new_매출액증가율,new_비유동자산증가율,감사의견_y,자본잠식률_100,자본잠식률_50
70560,(주)포스코건설,10000,2012-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c
70561,(주)포스코건설,10000,2013-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-71.478710,40.756393,-18.258059,13.631720,-16.164419,14.016854,4.081387,a,b,c
70562,(주)포스코건설,10000,2014-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-86.472926,5.344684,-12.449193,-31.415835,-8.092776,-6.396890,17.867219,a,b,c
70563,(주)포스코건설,10000,2015-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,127.038293,-4.509898,6.351912,-11.590310,-34.770304,-13.012219,-1.042386,a,b,c
70564,(주)포스코건설,10000,2016-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-2616.490936,-10.676382,-5.085581,-173.802637,54.677241,-15.921085,-8.134513,a,b,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221455,주식회사두루안,101635,2021-12-01,UQ,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,317.304431,-1.374490,58.085287,-7.162331,7.322329,1.872254,-1.302387,a,b,c
221456,주식회사두루안,101635,2022-12-01,UQ,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,-51.677152,54.399781,10.261314,-403.105893,63.082647,-12.227925,27.676594,a,b,c
16765,넬코리아주식회사,101667,2020-12-01,NaN,64102,토목 건설업,2017/08/08,20.0,중소기업,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,B,c
16766,넬코리아주식회사,101667,2021-12-01,QS,64102,토목 건설업,2017/08/08,20.0,중소기업,41,...,190.228461,13.969112,-23.104733,173.681224,-28.716629,-14.067465,64.013882,A,B,C


In [38]:
# Step 4: 
def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['자본잠식률_100'] == 'B']['회계년도'].min()
    
    # default 값이 nan이 아니면 작거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] <= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df_filtered = df_sorted.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

df_filtered.head()


,회사명,거래소코드,회계년도,감사의견코드,산업코드,산업명,설립일,기업규모코드,기업규모명,통계청 한국표준산업분류 코드 10차(대분류),...,new_순이익증가율,new_유형자산증가율,new_유동자산증가율,new_영업이익증가율,new_재고자산증가율,new_매출액증가율,new_비유동자산증가율,감사의견_y,자본잠식률_100,자본잠식률_50
0,(주)포스코건설,10000,2012-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c
1,(주)포스코건설,10000,2013-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-71.478710,40.756393,-18.258059,13.631720,-16.164419,14.016854,4.081387,a,b,c
2,(주)포스코건설,10000,2014-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-86.472926,5.344684,-12.449193,-31.415835,-8.092776,-6.396890,17.867219,a,b,c
3,(주)포스코건설,10000,2015-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,127.038293,-4.509898,6.351912,-11.590310,-34.770304,-13.012219,-1.042386,a,b,c
4,(주)포스코건설,10000,2016-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-2616.490936,-10.676382,-5.085581,-173.802637,54.677241,-15.921085,-8.134513,a,b,c


In [39]:
# Step 4: 
def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['자본잠식률_50'] == 'C']['회계년도'].min()
    
    # default 값이 nan이 아니면 작거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] <= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df_filtered_1 = df_filtered.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

df_filtered_1

,회사명,거래소코드,회계년도,감사의견코드,산업코드,산업명,설립일,기업규모코드,기업규모명,통계청 한국표준산업분류 코드 10차(대분류),...,new_순이익증가율,new_유형자산증가율,new_유동자산증가율,new_영업이익증가율,new_재고자산증가율,new_매출액증가율,new_비유동자산증가율,감사의견_y,자본잠식률_100,자본잠식률_50
0,(주)포스코건설,10000,2012-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c
1,(주)포스코건설,10000,2013-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-71.478710,40.756393,-18.258059,13.631720,-16.164419,14.016854,4.081387,a,b,c
2,(주)포스코건설,10000,2014-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-86.472926,5.344684,-12.449193,-31.415835,-8.092776,-6.396890,17.867219,a,b,c
3,(주)포스코건설,10000,2015-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,127.038293,-4.509898,6.351912,-11.590310,-34.770304,-13.012219,-1.042386,a,b,c
4,(주)포스코건설,10000,2016-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-2616.490936,-10.676382,-5.085581,-173.802637,54.677241,-15.921085,-8.134513,a,b,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206819,주식회사밸리온,101563,2022-12-01,UQ,137103,광고업,2011/07/29,0.0,NaN,71,...,-187.186038,-99.996809,38.874925,-5290.426318,NaN,180.772919,46.174475,a,b,c
206820,주식회사두루안,101635,2020-12-01,NaN,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c
206821,주식회사두루안,101635,2021-12-01,UQ,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,317.304431,-1.374490,58.085287,-7.162331,7.322329,1.872254,-1.302387,a,b,c
206822,주식회사두루안,101635,2022-12-01,UQ,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,-51.677152,54.399781,10.261314,-403.105893,63.082647,-12.227925,27.676594,a,b,c


In [40]:
# Step 4: 
def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['감사의견_y'] == 'A']['회계년도'].min()
    
    # default 값이 nan이 아니면 작거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] <= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df_filtered_2 = df_filtered_1.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

df_filtered_2.head()

,회사명,거래소코드,회계년도,감사의견코드,산업코드,산업명,설립일,기업규모코드,기업규모명,통계청 한국표준산업분류 코드 10차(대분류),...,new_순이익증가율,new_유형자산증가율,new_유동자산증가율,new_영업이익증가율,new_재고자산증가율,new_매출액증가율,new_비유동자산증가율,감사의견_y,자본잠식률_100,자본잠식률_50
0,(주)포스코건설,10000,2012-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c
1,(주)포스코건설,10000,2013-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-71.478710,40.756393,-18.258059,13.631720,-16.164419,14.016854,4.081387,a,b,c
2,(주)포스코건설,10000,2014-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-86.472926,5.344684,-12.449193,-31.415835,-8.092776,-6.396890,17.867219,a,b,c
3,(주)포스코건설,10000,2015-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,127.038293,-4.509898,6.351912,-11.590310,-34.770304,-13.012219,-1.042386,a,b,c
4,(주)포스코건설,10000,2016-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-2616.490936,-10.676382,-5.085581,-173.802637,54.677241,-15.921085,-8.134513,a,b,c


In [41]:
df_filtered_2[(df_filtered_2['자본잠식률_50'] == 'C')|(df_filtered_2['자본잠식률_100'] == 'B')|(df_filtered_2['감사의견_y'] == 'A')][['회사명','회계년도','이자보상배율','영업이익(손실)','자본잠식률','이자비용','감사의견_y','자본잠식률_100','자본잠식률_50']]

,회사명,회계년도,이자보상배율,영업이익(손실),자본잠식률,이자비용,감사의견_y,자본잠식률_100,자본잠식률_50
22,국제종합기계(주),2012-12-01,-160.665354,-11662110.0,107.573643,7258634.0,a,B,c
134,대한상선주식회사,2012-12-01,-152.891890,-35467986.0,432.402859,23198082.0,a,B,c
220,글로웨이(주),2015-12-01,-948.655837,-24459516.0,1007.495312,2578334.0,A,B,c
288,주식회사케이티엔지니어링,2013-12-01,1188.781769,16307423.0,-287.674870,1371776.0,A,b,c
413,프라임개발(주),2012-12-01,36.759926,14912450.0,545.449981,40567138.0,a,B,c
...,...,...,...,...,...,...,...,...,...
177560,에너콘코리아유한회사,2020-12-01,-1082.387689,-1377912.0,654.666139,127303.0,a,B,c
177564,주식회사팩스,2020-12-01,-200118.983957,-1496890.0,285.624000,748.0,a,B,c
177568,주식회사정진애드,2020-12-01,-624.032018,-1159283.0,423.069032,185773.0,a,B,c
177570,주식회사제이케이글로벌,2021-12-01,217.974930,415785.0,-205.056327,190749.0,A,b,c


In [42]:
df_filtered_2

,회사명,거래소코드,회계년도,감사의견코드,산업코드,산업명,설립일,기업규모코드,기업규모명,통계청 한국표준산업분류 코드 10차(대분류),...,new_순이익증가율,new_유형자산증가율,new_유동자산증가율,new_영업이익증가율,new_재고자산증가율,new_매출액증가율,new_비유동자산증가율,감사의견_y,자본잠식률_100,자본잠식률_50
0,(주)포스코건설,10000,2012-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c
1,(주)포스코건설,10000,2013-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-71.478710,40.756393,-18.258059,13.631720,-16.164419,14.016854,4.081387,a,b,c
2,(주)포스코건설,10000,2014-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-86.472926,5.344684,-12.449193,-31.415835,-8.092776,-6.396890,17.867219,a,b,c
3,(주)포스코건설,10000,2015-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,127.038293,-4.509898,6.351912,-11.590310,-34.770304,-13.012219,-1.042386,a,b,c
4,(주)포스코건설,10000,2016-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-2616.490936,-10.676382,-5.085581,-173.802637,54.677241,-15.921085,-8.134513,a,b,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177573,주식회사밸리온,101563,2022-12-01,UQ,137103,광고업,2011/07/29,0.0,NaN,71,...,-187.186038,-99.996809,38.874925,-5290.426318,NaN,180.772919,46.174475,a,b,c
177574,주식회사두루안,101635,2020-12-01,NaN,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c
177575,주식회사두루안,101635,2021-12-01,UQ,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,317.304431,-1.374490,58.085287,-7.162331,7.322329,1.872254,-1.302387,a,b,c
177576,주식회사두루안,101635,2022-12-01,UQ,105802,소프트웨어 개발 및 공급업,2007/10/15,20.0,중소기업,58,...,-51.677152,54.399781,10.261314,-403.105893,63.082647,-12.227925,27.676594,a,b,c


In [43]:
df_filtered_2[df_filtered_2['이자보상배율'].isnull() == True][(df_filtered_2['자본잠식률_50'] == 'C')|(df_filtered_2['자본잠식률_100'] == 'B')|(df_filtered_2['감사의견_y'] == 'A')][['회사명','회계년도','이자보상배율','영업이익(손실)','이자비용','감사의견_y','자본잠식률_100','자본잠식률_50']]

C:\Users\hotte\AppData\Local\Temp\ipykernel_19332\298702462.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered_2[df_filtered_2['이자보상배율'].isnull() == True][(df_filtered_2['자본잠식률_50'] == 'C')|(df_filtered_2['자본잠식률_100'] == 'B')|(df_filtered_2['감사의견_y'] == 'A')][['회사명','회계년도','이자보상배율','영업이익(손실)','이자비용','감사의견_y','자본잠식률_100','자본잠식률_50']]


,회사명,회계년도,이자보상배율,영업이익(손실),이자비용,감사의견_y,자본잠식률_100,자본잠식률_50
53038,삼척온천주식회사,2019-12-01,NaN,0.0,0.0,a,B,c
61737,디더블유개발주식회사,2014-12-01,NaN,0.0,0.0,A,B,c
72122,주식회사몬디케이에스피,2021-12-01,NaN,0.0,0.0,a,B,c
88192,대유SE,2018-12-01,NaN,0.0,0.0,A,b,C
131935,파크호텔주식회사,2022-12-01,NaN,0.0,0.0,A,b,c
168930,세종에스투유한회사,2020-12-01,NaN,0.0,0.0,A,b,c
177544,(주)황지,2021-12-01,NaN,NaN,NaN,A,b,c


In [44]:
df_filtered_2.to_csv('../dataset2/data2A_첫행날리기전_라벨링하고부도이후날림.csv')